**# Crash-Recovery beim SQL Server-Start**  

## Einführung  
Wenn ein SQL Server neu gestartet wird oder eine Datenbank aus einem Backup wiederhergestellt wird, erscheint häufig der Zustand *“In Recovery”*. Dies bedeutet, dass SQL Server gerade automatisch eine Wiederherstellung der Datenbank durchführt, um deren Konsistenz sicherzustellen【10†L190-L198】. Während dieser *Crash-Recovery*-Phase werden unvollständige Transaktionen – also Änderungen, die zum Zeitpunkt des Absturzes noch nicht abgeschlossen waren – entweder nachträglich ausgeführt oder rückgängig gemacht. Dadurch wird sichergestellt, dass die Datenbank nach dem Start keinen inkonsistenten Zustand aufweist und keine teilweisen Transaktionen sichtbar sind【10†L162-L170】. Der Recovery-Vorgang läuft automatisch im Hintergrund ab und erfordert in der Regel kein manuelles Eingreifen („This is an informational message only. No user action is required.“). Allerdings kann es Administratoren überraschen, wenn eine Datenbank ungewöhnlich lange im Recovery-Modus bleibt, da in dieser Zeit kein Zugriff auf die betroffene Datenbank möglich ist. Im Folgenden wird erläutert, **wann** und **warum** es zu einer solchen Recovery-Phase kommt, **was dabei im Hintergrund abläuft**, wie man **erkennen** kann, welche Datenbank sich im Recovery befindet, und wie man den **Fortschritt überwachen** kann. Zudem geben wir Hinweise, wovon die **Dauer** der Recovery abhängt und wie man lange Recovery-Zeiten vermeiden kann. Die Erklärungen richten sich an Leser ohne Vorkenntnisse ebenso wie an erfahrene DBAs, für die auch Skripte und tiefere Einblicke enthalten sind.  

## Wann gerät eine Datenbank in den Recovery-Modus?  
Eine SQL-Server-Datenbank durchläuft immer dann eine Crash-Recovery, wenn sie nicht sauber heruntergefahren wurde oder wenn Änderungen nachzuholen sind. Typische Szenarien sind:  

- **Unerwarteter Server-Absturz oder Neustart:** Bei Stromausfall, Systemabsturz, Bluescreen oder einem „harten“ Neustart des Servers wird SQL Server nicht ordnungsgemäß heruntergefahren. Beim nächsten Start des SQL Server-Dienstes befinden sich alle Datenbanken, die zum Zeitpunkt des Absturzes aktiv waren, zunächst im Recovery-Modus【10†L190-L198】. Der Server meldet im Fehlerprotokoll z.B.: `Starting up database 'XYZ'`. Anschließend beginnt für jede Datenbank die Wiederherstellung.  
- **Abbrechen langer Transaktionen:** Wenn ein DBA oder das System selbst einen laufenden Prozess abrupt beendet (z.B. durch `KILL` eines Sessions oder einen erzwungenen Neustart, während Transaktionen laufen), können unvollendete Transaktionen vorliegen. Beim nächsten Start der Datenbank müssen diese zurückgerollt werden. Zum Beispiel: Wenn ein großes DELETE-Statement mit Millionen von Datensätzen mitten in der Ausführung gestoppt wird, **muss** der SQL Server beim Start alle schon erfolgten Änderungen dieser Transaktion wieder rückgängig machen. Solche Situationen führen oft zu längeren Recovery-Zeiten【3†L47-L54】. Ein Erfahrungswert: das Zurückrollen (Undo) einer Transaktion dauert ungefähr so lange wie die Transaktion selbst gedauert hat – oder länger【3†L47-L54】.  
- **Wiederherstellung eines Backups:** Auch beim **Restore** eines Datenbank-Backups (mit **RECOVERY**) durchläuft die Datenbank intern eine Recovery-Phase, bevor sie benutzbar wird【10†L190-L198】. Solange ein Restore nicht abgeschlossen ist, zeigt die Datenbank den Status *Restoring*. Nach Abschluss des Einspielens der Backup-Dateien findet die eigentliche Recovery statt, um das wiederhergestellte Backup in einen konsistenten Zustand zu bringen.  
- **Datenbank-Attach oder Datei-Wiederherstellung:** Wenn eine MDF/NDF-Datei angehängt (*attach*) wird oder eine offline genommene Datenbank wieder online geschaltet wird, kann ebenfalls eine Recovery nötig sein. Insbesondere wenn die Datei nicht sauber getrennt wurde, liest SQL Server beim Attach zunächst das Transaktionslog der Datenbank und vollzieht notwendige Redo/Undo-Schritte, als wäre es ein Crash-Recovery.  
- **Geplantes Herunterfahren ohne Checkpoint:** Normalerweise führt SQL Server bei einem regulären **Herunterfahren** (z.B. `SHUTDOWN` oder Stoppen des Dienstes) automatisch einen letzten **Checkpoint** in jeder Datenbank durch. Ein Checkpoint stellt sicher, dass die meisten geänderten Seiten aus dem Speicher auf die Datendateien geschrieben und Transaktionslog-Einträge gesichert wurden. Dadurch ist beim nächsten Start kaum Recovery nötig. Falls jedoch ein Dienst-Stop erzwungen wird (z.B. Timeout beim Herunterfahren) oder der Server sofort beendet wird, kann es sein, dass nicht für alle Transaktionen ein Checkpoint durchgeführt wurde. Diese Datenbanken müssen beim nächsten Start trotzdem eine Crash-Recovery durchführen, obwohl der Neustart geplant war.  

In all diesen Fällen sieht man, dass eine Datenbank den Zustand **RECOVERING** annimmt. Laut Microsoft-Dokumentation bedeutet *RECOVERING*, dass die Datenbank gerade wiederhergestellt wird; dieser Zustand ist vorübergehend – sofern die Recovery erfolgreich verläuft, wechselt die DB anschließend automatisch auf ONLINE, andernfalls auf SUSPECT【18†L71-L79】. Während *RECOVERING* ist die Datenbank nicht für Benutzer zugänglich【18†L71-L78】. In der Systemtabelle `sys.databases` kann man den Status jeder Datenbank prüfen; ein Status von **RECOVERING** oder **RECOVERY_PENDING** (siehe unten) weist darauf hin, dass die DB noch im Wiederherstellungsprozess steckt【20†L350-L358】.  

> **Hinweis:** Es gibt einen Unterschied zwischen dem Status **RECOVERING** und **RECOVERY_PENDING**【18†L75-L83】. *RECOVERING* bedeutet, dass die automatische Wiederherstellung läuft. *RECOVERY_PENDING* hingegen zeigt an, dass die Wiederherstellung zwar nötig ist, aber nicht gestartet werden kann – oft wegen eines fehlenden oder defekten Logfiles oder Ressourcenproblemen【18†L75-L83】. In diesem Fall muss der DBA zunächst die Ursache beheben, bevor SQL Server die Recovery fortsetzen (oder die DB als SUSPECT markieren) kann.  

## Was passiert während der Crash-Recovery? (Hintergrund)  
Während der Recovery-Phase durchläuft SQL Server **drei Schritte**, um die Datenbank in einen konsistenten Zustand zu versetzen【10†L212-L220】【10†L224-L231】:  

1. **Analysis (Analyse-Phase):** Zunächst analysiert SQL Server das Transaktionslog ab dem letzten gültigen **Checkpoint** bis zum Ende des Logs【10†L216-L224】. Dabei werden zwei interne Tabellen erstellt: Die **Dirty Page Table (DPT)**, welche alle Datenbank-Seiten aufführt, die zum Zeitpunkt des Absturzes im Speicher modifiziert waren und noch nicht auf Disk gespeichert wurden („dirty pages“), sowie die **Active Transaction Table (ATT)** mit allen Transaktionen, die zum Zeitpunkt des Crashs noch *nicht* abgeschlossen (uncommitted) waren【10†L218-L226】. Diese Analyse ermittelt also, welche Änderungen eventuell nachzuholen sind und welche Transaktionen zurückgerollt werden müssen.  
2. **Redo (Nachholen der Änderungen):** In der Redo-Phase spielt SQL Server alle im Log seit dem letzten Checkpoint protokollierten **committeten** (festgeschriebenen) Änderungen erneut durch【10†L224-L232】. Konkret bedeutet das: Alle Transaktionen, die vor dem Absturz **bereits erfolgreich abgeschlossen** waren, deren Änderungen aber möglicherweise noch nicht vollständig in die Datendateien geschrieben wurden, werden nun nachgetragen. Man spricht von *roll forward*: Das Datenbank-Dateisystem wird auf den neuesten konsistenten Stand gebracht, indem alle fehlenden Änderungen angewendet werden. Am Ende der Redo-Phase sind somit **alle bestätigten Transaktionen im Datenbestand nachgezogen**. **Wichtig:** Sobald Redo abgeschlossen ist, kann die Datenbank in vielen Fällen bereits für Lesezugriffe geöffnet werden (siehe „fast recovery“ weiter unten)【8†L87-L94】.  
3. **Undo (Rückgängig-Phase):** In der Undo-Phase werden alle **unvollendeten** Transaktionen, die im ATT als aktiv markiert waren, **zurückgerollt**【10†L231-L238】. Das heißt, alle Änderungen dieser nicht committeten Transaktionen, die möglicherweise teilweise bereits in den Datenbankseiten vorhanden sind, werden wieder entfernt. Dies stellt sicher, dass keine halbfertigen Transaktionen sichtbar bleiben. SQL Server macht dabei jeden einzelnen Schritt dieser abgebrochenen Transaktionen im Log rückgängig (*roll back*), als ob sofort ein `ROLLBACK` ausgeführt würde. Nachdem **Undo** abgeschlossen ist, gilt die Datenbank als vollständig konsistent und die Recovery ist beendet.  

Während dieser Phasen protokolliert SQL Server im Fehlerlog den Fortschritt. Beispielsweise sieht man Einträge wie:  

```text
Recovery of database 'MyDatabase' (10) is 55% complete (approximately 1066 seconds remain). Phase 3 of 3.
```  

*Beispiel einer Fehlermeldung im SQL Server-Errorlog während einer laufenden Recovery【21†L227-L234】.* Solche Meldungen helfen zu verstehen, in welcher Phase die Recovery ist (hier *Phase 3 von 3*, also die Undo-Phase) und wie weit sie fortgeschritten ist (55%). Außerdem schätzt SQL Server hier die verbleibende Dauer (ca. 1066 Sekunden, also ~18 Minuten) – diese Schätzung basiert auf dem bisherigen Fortschritt, kann aber ungenau sein.  

Am Ende der Recovery erscheinen Meldungen, die Zusammenfassen, was passiert ist, etwa: *“3807 transactions rolled forward... 0 transactions rolled back... Recovery completed for database 'XYZ' in 30 seconds (analysis 1289 ms, redo 29343 ms, undo 72 ms.)”*【15†L278-L287】【15†L288-L290】. Daraus kann man entnehmen, wie viele Transaktionen nachgezogen (redo) und zurückgerollt (undo) wurden und wie lange jede Phase gedauert hat.  

**Warum ist dieser Ablauf nötig?** SQL Server benutzt das Prinzip **Write-Ahead Logging (WAL)**: Alle Änderungen werden zuerst im Transaktionslog (`.ldf`) festgehalten, bevor sie in die Datenbankdateien (`.mdf`/`.ndf`) geschrieben werden. Daher kann SQL Server nach einem Absturz mithilfe des Logs rekonstruieren, welche Änderungen evtl. noch in den Datenbankseiten fehlen (Redo) und welche Transaktionen nicht abgeschlossen waren (Undo). Crash-Recovery garantiert so die *ACID*-Eigenschaften, insbesondere Konsistenz und Dauerhaftigkeit (Durability): Entweder sind Transaktionen vollständig ausgeführt oder vollständig zurückgerollt – ein „halb fertiger“ Zustand wird nie sichtbar.  

### Fast Recovery (Enterprise Edition)  
In SQL Server **Enterprise Edition** gibt es ein Feature namens **Fast Recovery**. Dieses ermöglicht es, eine Datenbank bereits **nach Abschluss der Redo-Phase (Phase 2)** online zu schalten, *noch bevor* die Undo-Phase vollständig fertig ist【8†L87-L94】. Das bedeutet: In Enterprise Edition können Benutzer eine Datenbank oft schon verwenden, während im Hintergrund noch die letzten Rollbacks laufen. Nur auf die Daten, die gerade zurückgerollt werden, kann in dieser Zeit nicht zugegriffen werden – sie bleiben bis zum Ende der Undo-Phase gesperrt【8†L87-L94】. In **Standard Edition** hingegen bleibt die Datenbank bis zum Abschluss **aller drei Phasen** offline (Status = *Recovery*)【5†L247-L254】. Dies erklärt, warum in Standard Edition die Downtime länger erscheint: die DB steht tatsächlich erst zur Verfügung, wenn Undo komplett abgeschlossen ist. Fast Recovery in Enterprise Edition verbessert also die Verfügbarkeit, insbesondere bei lang andauernden Rollbacks.  

### Accelerated Database Recovery (ADR)  
Seit **SQL Server 2019** gibt es eine optionale, grundlegend neue Recovery-Engine namens **Accelerated Database Recovery (ADR)**【19†L35-L43】. ADR verändert den internen Ablauf der Recovery: Zwar gibt es weiterhin die Phasen Analyse, Redo und Undo, doch für Undo wird ein anderer Ansatz verwendet. SQL Server führt dafür einen **Versionsspeicher** in der Datenbank ein, um Änderungen auf Zeilenebene nachzuverfolgen【19†L41-L49】. Uncommittete Transaktionen können dadurch nahezu *sofort* rückgängig gemacht werden, ohne das gesamte Log sequentiell abarbeiten zu müssen【19†L42-L49】. Das Ergebnis: Die Dauer der Crash-Recovery hängt nicht mehr von der Länge großer Transaktionen ab – Recovery wird wesentlich schneller und berechenbarer, selbst in hochtransaktionalen Umgebungen【19†L37-L45】. ADR muss pro Datenbank aktiviert werden (Datenbank-Eigenschaft *ACCELERATED_DATABASE_RECOVERY*), und ist ab SQL 2022 für Systemdatenbanken standardmäßig aktiv. Für DBAs bedeutet ADR ein mächtiges Werkzeug, um Downtimes durch lange Rollbacks drastisch zu reduzieren. Allerdings sollte man neue Features wie ADR zunächst ausgiebig testen, bevor man sie in Produktionsumgebungen aktiviert (Microsoft verbessert ADR laufend weiter, z.B. in SQL 2022/2025)【19†L51-L59】【19†L68-L76】.  

## Ermitteln, welche Datenbank im Recovery-Modus ist  
In einem SQL Server mit vielen Datenbanken möchte ein DBA oft schnell herausfinden, **welche** Datenbank gerade im Recovery-Prozess steckt – beispielsweise nach einem Server-Neustart. Folgende Methoden helfen dabei:  

- **SQL Server-Fehlerlog prüfen:** Das Errorlog protokolliert zu Beginn des Startvorgangs für jede Datenbank eine Meldung *“Starting up database 'XYZ'”*. Danach erscheinen für jede Datenbank im Recovery-Modus Meldungen wie *“Recovery of database 'XYZ' (ID) is X% complete”*. Anhand dieser Einträge erkennt man sofort, welche DBs gerade wiederhergestellt werden und wie weit sie sind. In SQL Server Management Studio (SSMS) kann man unter **Verwaltung > SQL Server-Protokolle > aktuelles Log** diese Meldungen einsehen【21†L221-L229】【21†L227-L234】. Alternativ lässt sich das Errorlog auch per T-SQL abfragen, z.B. mit dem System-SPROC `sp_readerrorlog`. Ein Beispiel:  

  ```sql
  -- Zeigt alle Einträge im aktuellen Errorlog, die "Recovery of database" enthalten
  EXEC sp_readerrorlog 0, 1, 'Recovery of database';
  ```  

  Dieses Skript listet alle Datenbanken auf, für die es Wiederherstellungs-Meldungen gibt, inklusive Prozentangaben. Man kann auch einen bestimmten Datenbanknamen als vierten Parameter angeben, um nur Einträge für diese DB zu sehen.  

- **Datenbank-Status via DMV abfragen:** Über die Katalogsicht `sys.databases` kann man den Status aller DBs abfragen. Beispielsweise liefert folgendes Statement alle Datenbanken, die nicht **ONLINE** sind:  

  ```sql
  SELECT name, state_desc  
  FROM sys.databases  
  WHERE state_desc <> 'ONLINE';
  ```  

  Wenn hier eine Datenbank als **RECOVERING** erscheint, bedeutet dies, dass sie gerade im Recovery-Modus ist【20†L350-L358】. Eine neu startende Datenbank kann auch vorübergehend den Status **RESTORING** haben, was in diesem Kontext ähnlich zu interpretieren ist (die Datenbank ist noch nicht online, entweder wegen Restore oder Recovery). Der Status **RECOVERY_PENDING** weist auf ein Problem hin (Recovery wartet, siehe oben). Im Normalfall sollte der Status *RECOVERING* nach Abschluss der Recovery auf *ONLINE* umspringen; bleibt eine DB hängen, wechselt sie u.U. auf *SUSPECT*.  

- **Aktive Prozesse überprüfen (sp_who2 / sys.dm_exec_requests):** Während der Recovery wird im Hintergrund ein Systemprozess ausgeführt. Mit dem altbekannten DBA-Kommando `EXEC sp_who2` kann man alle Sessions auflisten. Hier taucht ein Systemprozess mit dem **Command** `DB STARTUP` auf – dieser gehört zur Recovery der jeweiligen Datenbank【3†L79-L87】. An der **DBID** bzw. der zugehörigen **Database**-Spalte in `sp_who2` erkennt man, für welche Datenbank dieser Recovery-Prozess läuft. (In der Regel gibt es pro instanz gleichzeitig nur wenige Recovery-Prozesse – oft jeweils einen pro Datenbank, die gerade wiederhergestellt wird.) Moderne Methoden nutzen lieber Dynamic Management Views: In der View `sys.dm_exec_requests` findet man ebenfalls einen Eintrag, dessen `command`-Spalte den Wert `'DB STARTUP'` hat【5†L221-L228】. Darüber kann man z.B. so die betroffene Datenbank identifizieren:  

  ```sql
  SELECT DB_NAME(database_id) AS DatabaseName, command, status, percent_complete  
  FROM sys.dm_exec_requests  
  WHERE command = 'DB STARTUP';
  ```  

  Dieses Statement liefert – falls gerade ein Recovery läuft – die DatabaseName und den Fortschritt (`percent_complete`) des Recovery-Vorgangs. Ist kein Ergebnis zurückgegeben, befindet sich aktuell keine DB im Status Recovery (zumindest nicht in Phase 2 oder 3). Beachte: `percent_complete` kann anfangs 0.0 anzeigen und erst gegen Ende hochspringen; in vielen Fällen ist diese DMV nicht so informativ wie die Logs【21†L289-L297】. Dennoch sieht man hier, ob der Prozess noch aktiv ist und ob er eventuell blockiert (Spalte `status` zeigt z.B. „suspended“, falls Wartezeiten auftreten).  

**Tipp:** Auch SQL Server Management Studio zeigt im Objekt-Explorer manchmal Hinweise. Wenn man kurz nach dem Start die Liste der Datenbanken betrachtet, kann bei einer Datenbank `[Recovery]` oder `[Restoring]` stehen. Oft wird allerdings einfach nur die Datenbank nicht zugreifbar sein, bis sie online kommt – beim Versuch, sie zu expandieren, erhält man eine Fehlermeldung, solange sie im Recovery-Modus ist. Daher sind die obigen Methoden (Log oder DMV) zuverlässiger, um gezielt nachzusehen.  

## Fortschritt des Recovery-Prozesses überwachen  
Gerade bei größeren Datenbanken möchte man wissen, wie weit die Recovery fortgeschritten ist und wie lange sie noch dauern könnte. SQL Server bietet hier begrenzte Einblicke, hauptsächlich über das Fehlerprotokoll und teilweise über DMVs:  

- **Fortschrittsmeldungen im Errorlog:** Wie oben erwähnt, protokolliert SQL Server während der Recovery in gewissen Abständen Einträge mit **Prozentangaben** und einer **geschätzten Restzeit**. Diese Meldungen lauten z.B.: *“Recovery of database 'XYZ' (9) is 5% complete (approximately 85 seconds remain). Phase 2 of 3.”*【15†L263-L271】. Darin ist die aktuelle Phase (1, 2 oder 3) und der Fortschritt ersichtlich. Die Frequenz dieser Meldungen hängt von der Dauer der Recovery ab – bei sehr schneller Recovery (sekundenbruchteile) sieht man ggf. keine Zwischenmeldungen, bei längeren Recovery-Prozessen (Minuten/Stunden) schreibt SQL Server regelmäßig Updates ins Log. Anhand der Prozentangabe und Sekundenabschätzung kann man den Fortschritt gut nachvollziehen. Beachten Sie jedoch, dass die Zeitschätzung nur ein grober Richtwert ist („approximately... remain“) und sich während des Prozesses auch wieder ändern kann.  

- **Abfrage per `sys.dm_exec_requests`:** Die genannte DMV zeigt für den `DB STARTUP`-Prozess ebenfalls eine Fortschrittsanzeige. Die Spalte `percent_complete` gibt den Fortschritt in Prozent und `estimated_completion_time` (in Millisekunden) die voraussichtlich verbleibende Zeit an【3†L143-L151】【3†L169-L172】. Ein Beispiel (ähnlich obigem Query):  

  ```sql
  SELECT DB_NAME(database_id) AS DatabaseName,
         percent_complete,
         estimated_completion_time,
         status
  FROM sys.dm_exec_requests
  WHERE command = 'DB STARTUP';
  ```  

  Die `estimated_completion_time` liefert die Restdauer in Millisekunden; man kann sie z.B. durch 1000*60 teilen, um Minuten zu erhalten. Diese Werte korrespondieren mit den Angaben im Errorlog【3†L169-L177】. Allerdings ist bekannt, dass `percent_complete` bei Recovery-Vorgängen manchmal lange auf 0% stehen bleibt und erst spät aktualisiert wird【21†L289-L297】 – verlassen Sie sich also lieber auf die Log-Meldungen.  

- **SQL Server Management Studio UI:** In neueren SSMS-Versionen kann man über den Logfile-Viewer (s. o.) komfortabel die aktuellen Recovery-Meldungen einsehen【21†L221-L229】. Es gibt allerdings *kein* spezielles GUI-Fenster, das den Recovery-Fortschritt in Form eines Balkens oder ähnlichem anzeigt – man muss tatsächlich in die Log-Einträge schauen.  

Wenn man ein automatisiertes Monitoring aufbauen möchte, könnte man periodisch `xp_readerrorlog` oder `sp_readerrorlog` aufrufen, um nach bestimmten Mustern wie *“is X% complete”* zu filtern, und diese Ergebnisse z.B. per Skript auswerten. Ein DBA-Skript von MSSQLTips ermöglicht es beispielsweise, das Errorlog nach der neuesten Fortschrittsmeldung einer bestimmten Datenbank zu durchsuchen und daraus Prozent sowie geschätzte Restzeit zu extrahieren【1†L155-L164】【1†L227-L234】. Solche Lösungen lesen die Textmeldung und parsen die Zahlen heraus – das kann hilfreich sein, wenn man bei sehr langen Recovery-Prozessen regelmäßig über den Stand informiert werden möchte (z.B. Alerting alle 30 Minuten). Für die meisten Fälle genügt jedoch ein Blick ins SQL Server-Log oder die einmalige Abfrage der DMV.  

**Kann man die Recovery beschleunigen oder abbrechen?** In der Regel **nein** – der Recovery-Vorgang läuft so schnell wie möglich ab, basierend auf I/O-Leistung und CPU. Es gibt keinen offiziellen Weg, ihn *sicher* zu unterbrechen oder zu überspringen. Ein Neustart des SQL Server-Dienstes würde die Recovery **nicht** abbrechen, sondern beim nächsten Start **von vorn beginnen**【5†L214-L222】【5†L243-L250】, was meist nur weiteren Zeitverlust bedeutet. Daher sollte man einen laufenden Recovery-Prozess im Normalfall **weiterlaufen lassen**, bis er abgeschlossen ist.  

## Wie lange dauert eine Crash-Recovery?  
Die Dauer des Recovery-Prozesses kann von ein paar Sekunden bis zu mehreren Stunden (in Extremfällen sogar Tagen) reichen, je nach Situation. Wichtige Einflussfaktoren sind:  

- **Größe des Transaktionslogs und letzte Checkpoint-Historie:** SQL Server muss das Log seit dem letzten Checkpoint durchgehen. Ist das Logfile sehr groß (viele nicht persistierte Änderungen), dauert die Analyse und das Redo entsprechend länger. Insbesondere wenn die Datenbank im **Simple Recovery Mode** sehr große Volumes an Transaktionen seit dem letzten automatischen Checkpoint hatte, kann das Log große Mengen an Wiederholungsarbeit beinhalten.  
- **Anzahl und Größe unvollendeter Transaktionen:** Ein Hauptfaktor ist, ob zum Crash-Zeitpunkt **lange Transaktionen** liefen. Ein langer laufender Batch, der abgebrochen wurde, muss komplett zurückgerollt werden. Wie erwähnt, kann ein Rollback so lange oder länger dauern wie die ursprüngliche Transaktion【3†L47-L54】. Zudem werden Rollbacks **single-threaded** durchgeführt, während die ursprüngliche Transaktion evtl. parallelisiert war – dies kann die Undo-Phase deutlich verlangsamen【5†L209-L218】. Beispiel: Eine massive Update-Operation mit Parallelität auf mehreren Kernen bricht ab – der Undo läuft dann seriell auf einem Kern durch und benötigt entsprechend viel Zeit.  
- **Datenbankgröße und I/O-Leistung:** Das Nachholen committeter Änderungen (Redo) erfordert das Lesen des Logs und Schreiben in die Datendateien. Bei einer großen Datenbank mit vielen *Dirty Pages* zum Zeitpunkt des Crashs müssen entsprechend viele Seiten neu geschrieben werden. Die Geschwindigkeit der Storage-(Platten-)I/O und evtl. bestehende **Disk- oder IO-Engpässe** beeinflussen die Dauer stark【3†L55-L63】【3†L56-L59】. Langsame oder ausgelastete Speichersysteme verlängern die Recovery erheblich.  
- **Virtual Log Files (VLFs):** Das Transaktionslog ist intern in sogenannte VLF-Segmente unterteilt. Eine extreme Anzahl an VLFs (z.B. durch häufiges kleinschrittiges Autogrowth des Logs) kann die Recovery verlangsamen【15†L301-L304】. SQL Server muss unter Umständen viele kleine VLFs sequentiell abarbeiten. Es gibt Berichte, dass sehr viele VLFs den Recovery-Prozess merklich ausbremsen können【5†L229-L237】.  
- **Instant File Initialization (IFI):** IFI beschleunigt das Erstellen und Erweitern von Datendateien, indem sie nicht mit Nullen initialisiert werden müssen. Bei der Crash-Recovery kann IFI ins Spiel kommen, wenn z.B. der TempDB neu angelegt wird oder wenn während Redo eine Vergrößerung der Datenbankdatei, die vor dem Absturz angestoßen wurde, fortgesetzt werden muss. Ist dem SQL-Dienstkonto das Recht *Perform Volume Maintenance Tasks* nicht gewährt (also IFI **nicht** aktiv), kann das in bestimmten Fällen die Recovery verlangsamen【5†L229-L237】 (weil dann evtl. doch null-init durchgeführt werden muss, z.B. für TempDB oder beim Zuwachs von Dateien).  
- **SQL Server-Edition und -Features:** Wie oben beschrieben, erlaubt die Enterprise Edition ein früheres Online-Gehen durch Fast Recovery, was effektiv die wahrgenommene Ausfallzeit reduziert【5†L247-L254】. Moderne Versionen mit ADR vermeiden lange Undo-Zeiten weitgehend【19†L37-L45】. In älteren Versionen (SQL 2005/2008) gab es teils Bugs, die ungewöhnlich lange Recovery-Zeiten verursachen konnten, welche per Hotfix behoben wurden【15†L310-L318】.  

Man sollte sich bewusst sein, dass **jede Transaktion, die zum Zeitpunkt des Absturzes noch offen war, die Recovery verlängern kann**. Daher gilt: Je sauberer und kürzer die Transaktionen üblicherweise sind, desto schneller verläuft im Worst Case die Wiederherstellung.  

## Best Practices, um lange Recovery-Zeiten zu vermeiden  
Es gibt einige Maßnahmen, die DBAs treffen können, um im Falle eines Crashs die Recovery-Dauer möglichst kurz zu halten bzw. große Überraschungen zu vermeiden:  

- **Regelmäßige Checkpoints / Indirect Checkpoints:** Standardmäßig steuert SQL Server die Checkpoints automatisch abhängig von der Workload. In überwiegend OLTP-Systemen kann man die *Target Recovery Time* für eine Datenbank konfigurieren, um indirekte Checkpoints zu aktivieren【8†L261-L269】. Zum Beispiel: `ALTER DATABASE [MyDB] SET TARGET_RECOVERY_TIME = 60 SECONDS;`. Dadurch versucht SQL Server sicherzustellen, dass eine Crash-Recovery nicht länger als 60 Sekunden dauern würde. Intern wird dann häufiger ein Checkpoint ausgeführt, um das In-Memory-Delta klein zu halten. Achtung: Ein zu kurzer Wert kann Overhead im laufenden Betrieb erzeugen (mehr I/O für Checkpoints)【8†L261-L269】. Für sehr kritische DBs kann dies aber sinnvoll sein.  
- **Transaktionslog richtig dimensionieren und VLFs minimieren:** Stellen Sie ausreichend große Transaktionslog-Dateien ein und vermeiden Sie häufiges automatisches Wachstum in kleinen Schritten. Ein Log, das z.B. in 10MB-Schritten tausendfach gewachsen ist, hat sehr viele VLFs. Besser ist, von Anfang an eine größere Loggröße zu wählen oder in größeren Blöcken zu wachsen, um die Anzahl der VLF-Segmente gering zu halten. Weniger VLFs können die Recovery beschleunigen【15†L301-L304】【5†L229-L237】. Microsoft empfiehlt, das Log in mäßigen Stücken wachsen zu lassen (z.B. 500 MB oder 1 GB pro Schritt, je nach DB-Größe). Tools wie `DBCC LOGINFO` (bis SQL 2016) oder `sys.dm_db_log_info` (SQL 2017+) zeigen die VLF-Anzahl an.  
- **Lange Transaktionen vermeiden bzw. aufteilen:** Transaktionen sollten nach Möglichkeit kurz gehalten werden. Vermeiden Sie riesige Einzel-Transaktionen, die Millionen von Zeilen betreffen und sehr lange laufen. Falls solche Massenupdates nötig sind, überlegen Sie, ob sie in Batches mit Zwischenergebnissen oder Zwischen-Commits ausgeführt werden können. Dadurch reduziert sich im Fehlerfall das zurückzurollende Volumen. Zudem blockieren kürzere Transaktionen seltener das Log-Truncation, was insgesamt das Logwachstum zügelt.  
- **Vor geplanten Neustarts aktive Transaktionen prüfen:** Wenn ein Neustart eines SQL Servers ansteht (z.B. Wartungsarbeiten, Patch-Reboot), sollte ein DBA zuvor sicherstellen, dass keine lang laufenden Transaktionen aktiv sind. Man kann z.B. via `sys.dm_tran_active_transactions` oder einfach `sp_who2` bzw. `sp_whoisactive` schauen, ob Sessions mit sehr hohen Laufzeiten oder uncommitteten Änderungen existieren. Notfalls wartet man noch etwas oder beendet diese Transaktionen kontrolliert. So verhindert man, dass der Server-Neustart in eine langwierige Recovery läuft【8†L259-L268】.  
- **Instant File Initialization aktivieren:** Gewähren Sie dem SQL Server-Dienstkonto das Benutzerrecht **Perform Volume Maintenance Tasks** auf Windows. Dadurch kann SQL Server Datendateien schneller anlegen oder vergrößern, ohne sie vollständig mit Nullen zu füllen. Das verkürzt zwar primär Backup/Restore und Datei-Operationen, kann aber im Recovery-Fall (z.B. beim Neuaufbau der TempDB oder Fortsetzen eines Wachstums) ebenfalls Zeit sparen【5†L229-L237】. (Hinweis: IFI betrifft nicht das Logfile – Logfiles müssen immer null-initialisiert werden aus Sicherheitsgründen. Es betrifft nur `.mdf/.ndf` Dateien.)  
- **Aktuelle SQL Server-Versionen und Features nutzen:** Erwägen Sie ein Upgrade auf neuere Versionen, um von Verbesserungen zu profitieren. Zum Beispiel verringert **ADR in SQL 2019+** drastisch die Recovery-Zeiten bei langen Transaktionen, wie oben erklärt【19†L37-L45】. Auch **Fast Recovery (Enterprise Edition)** steigert die Verfügbarkeit. Halten Sie zudem Ihre SQL Server Instanz gepatcht; bekannte Bugs, die zu Hängern im Recovery führen, wurden in Service Packs oder CUs behoben【15†L310-L318】.  
- **Monitoring und Alerts:** Implementieren Sie ein Monitoring, das Sie benachrichtigt, wenn eine Datenbank ungewöhnlich lange im Recovery verbleibt. So können Sie zeitnah reagieren (z.B. Nutzer informieren, dass DB noch in Recovery ist, oder im Extremfall einen Wiederherstellungsplan einleiten). Ein SQL Agent Job könnte etwa beim Server-Start alle paar Minuten prüfen, ob noch Datenbanken im Status *RECOVERING* sind und ggf. Warnungen loggen oder mailen.  

## Verhalten während der Recovery: Do’s and Don’ts  
Wenn der Ernstfall eintritt und eine oder mehrere Datenbanken sich im Recovery-Modus befinden, sollten Administratoren Folgendes beachten:  

- **Geduld haben und Recovery abschließen lassen:** So schwer es fällt – in den meisten Fällen ist **Warten** die beste (und einzige) Option【15†L308-L316】【8†L273-L279】. Der SQL Server arbeitet daran, die DB wieder online zu bringen. Ein Abbruch (z.B. durch erneuten Neustart des Dienstes) erzwingt nur einen erneuten Recovery-Durchlauf von vorne【5†L214-L222】. Mehrfaches Neustarten kann die Situation verschlimmern und im Worst Case zu **Datenkorruption** führen【3†L47-L54】.  
- **System nicht zusätzlich belasten:** Während der Recovery sollte man möglichst keine ressourcenintensiven Operationen auf dem Server starten. Insbesondere auf die betroffene(n) Datenbank(en) kann man ohnehin nicht zugreifen, aber auch auf andere Datenbanken sollte man größere Aktionen vielleicht verschieben, bis alle Recoveries durch sind – einfach, um der laufenden Recovery maximal I/O und CPU zu gönnen.  
- **Kein FORCE RECOVERY erzwingen:** Es gibt im normalen Betrieb keine unterstützte Option, eine Recovery *zu überspringen*. Insbesondere sollte man **keinesfalls** versuchen, manuell an den Dateien zu manipulieren oder mit Undocumented-Flags (wie `ALTER DATABASE ... SET EMERGENCY` inmitten einer Recovery) herumzuspielen – das kann die Situation verschlechtern.  
- **Überwachen:** Behalten Sie die Fortschrittsmeldungen im Auge (siehe vorheriges Kapitel). So erkennen Sie, ob die Recovery noch voranschreitet. Solange die Prozentwerte steigen oder das Log neue „X% complete“ Zeilen ausgibt, arbeitet SQL Server noch daran. Wenn die Fortschrittsanzeige über sehr lange Zeit stagniert (und auch I/O auf den entsprechenden Dateien nahe Null ist), könnte das auf ein Problem hindeuten – etwa einen Deadlock der Recovery (extrem selten) oder einen externen Faktor (z.B. fehlender Storage-Zugriff).  
- **Notfallszenario:** Wenn eine Recovery **unzumutbar lange** dauert (z.B. viele Stunden bei einer mittelwichtigen Datenbank) und Geschäftsanforderungen es nicht erlauben zu warten, bleibt als letztes Mittel oft nur, die Datenbank aus einem **Backup** wiederherzustellen. Dabei geht natürlich jeder Fortschritt seit dem Backup verloren. Diese Abwägung muss man im Einzelfall treffen. Tools zur *Datenbankreparatur* (wie von Drittanbietern angeboten【15†L320-L330】) werden manchmal beworben, sind aber meist nur bei wirklicher Dateikorruption relevant. In einer echten Hänge-Situation sollte eher Microsoft Support konsultiert werden, bevor man zu solchen Mitteln greift. Oft ist aber der beste Rat tatsächlich: Abwarten, bis SQL Server die Recovery abgeschlossen hat【15†L343-L351】【8†L273-L279】.  

## Fazit  
Dass sich eine SQL Server-Datenbank beim Start im *Recovery-Modus* befindet, ist zunächst **kein Grund zur Panik**, sondern meist ein normales Verhalten nach einem unerwarteten Ereignis. SQL Server stellt damit sicher, dass Ihre Datenbanken in einem konsistenten Zustand hochfahren und keine unvollständigen Transaktionen zurückbleiben. Wir haben gesehen, dass dieser Prozess in **drei Phasen (Analysis, Redo, Undo)** abläuft, um das Transaktionslog auszuwerten und alle notwendigen Aktionen nachzuholen oder rückgängig zu machen【10†L216-L224】【10†L224-L232】. In vielen Fällen dauert das nur Sekunden bis wenige Minuten. Bei sehr großen oder stark beanspruchten Datenbanken – insbesondere wenn kurz vor dem Absturz lange Transaktionen liefen – kann es jedoch zu erheblichen Verzögerungen kommen. Hier heißt es, geduldig zu sein und den Fortschritt zu beobachten, etwa über die Log-Einträge mit Prozentangaben【15†L263-L271】.  

Für Anfänger ist wichtig zu verstehen: **„In Recovery“** bedeutet nicht, dass Daten verloren sind – im Gegenteil, SQL Server *schützt* Ihre Daten gerade. Als DBA lernt man mit der Zeit, solche Situationen einzuschätzen und durch Prävention (kurze Transaktionen, korrekt dimensionierte Logs, etc.) zu entschärfen. Moderne Features wie **Fast Recovery** in der Enterprise Edition【8†L87-L94】 oder **Accelerated Database Recovery (ADR)**【19†L37-L45】 können die Verfügbarkeit weiter erhöhen und Recovery-Zeiten drastisch reduzieren.  

Letztlich sollte man Crash-Recovery als integralen Bestandteil von SQL Server betrachten, der ohne Zutun des Benutzers abläuft: *“This is an informational message only. No user action is required.”* – in den allermeisten Fällen gilt genau das. Wenn eine Datenbank also mal beim Start *„wiederhergestellt wird“*, wissen Sie nun, warum das passiert, was dabei vor sich geht und wie Sie den Vorgang beobachten können. Im Zweifel gilt: lieber den Prozess sauber durchlaufen lassen und daraus lernen (z.B. lange laufende Transaktionen überdenken), als durch Übereifer die Konsistenz der Daten zu gefährden【5†L214-L222】【5†L243-L252】. Denn am Ende ist eine erfolgreiche Recovery immer noch der beste Weg, Ihre Datenbank ohne Verlust wieder online zu bekommen. 【8†L239-L248】【15†L343-L351】
